In [4]:
import matplotlib.pyplot as plt
plt.rcParams['axes.grid'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
import pandas as pd
import numpy as np
import seaborn as sns
import timeit
import scipy.stats as st
from joblib import Parallel, delayed

### Constraints:
- Demand should be satisfied during lead time: demand ≤ 𝜎 * lead time. This implies the safety stock should be enough to cover the demand variability during lead time.
- Lead time should be within the given lead time for each product: LT ≤ lead time
- Inventory should be sufficient to meet the demand: starting stock + ∑(𝑄) − ∑(demand) ≥ 0
- Order Quantity Constraints: minimum order quantity (𝑄 ≥ 20% * average demand), maximum order quantity (𝑄 ≤ 50% * average demand)
    

In [5]:
start = timeit.default_timer()

def calculate_safety_stock(max_daily_sales, max_lead_time, avg_daily_sales, avg_lead_time):
    safety_stock = (max_daily_sales * max_lead_time) - (avg_daily_sales * avg_lead_time)
    return safety_stock

summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785]
}

class Product:
    def __init__(self, i):
        self.mean = np.mean([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.sd = np.std([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.mean_demand_lead_time = summary['Mean Demand (Lead Time)'][i - 1]
        self.std_dev_demand_lead_time = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.expected_demand_lead_time = summary['Expected Demand (Lead Time)'][i - 1]

def daily_demand(mean, sd, probability):
    if np.random.uniform(0, 1) > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS(Q, product, review_period=30, z_score=1.65):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability
    demand_lead = product.expected_demand_lead_time  # Corrected attribute name

    """
    - Importance sampling parameters
    - Define a new distribution that emphasizes extreme demand values
    - For simplicity, we used a mixture of two normal distributions
    - One centered around the mean and another centered around a high demand value
    """
    high_demand_value = 2 * mean  
    weights = [0.8, 0.2]  
    means = [mean, high_demand_value]
    sds = [sd, sd]  

    # max_daily_sales and avg_daily_sales 
    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]  
    max_daily_sales = np.max(daily_sales)
    avg_daily_sales = np.mean(daily_sales)

    # max_lead_time and avg_lead_time
    max_lead_time = lead_time
    avg_lead_time = np.mean(summary['Lead Time'])

    # safety stock
    safety_stock = calculate_safety_stock(max_daily_sales, max_lead_time, avg_daily_sales, avg_lead_time)

   
    q = 0
    stock_out = 0
    counter = 0
    order_placed = False
    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': [], 'reorder_quantities': []}  # Added 'reorder_quantities'

    for day in range(1, 365):
        # Sample demand from the new distribution
        day_demand = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]  # Corrected list comprehension
        if day_demand != 0:  # Add this condition to skip printing when demand is zero
            data['daily_demand'].append(day_demand)

        if day % review_period == 0:
           
            q = max(0, safety_stock + demand_lead - inventory)
            q = max(q, int(0.2 * mean))  # Minimum order quantity
            order_placed = True
            data['orders'].append(q)
            data['reorder_quantities'].append(q)  # Store reorder quantity

        if order_placed:
            counter += 1

        if counter == lead_time:
            inventory += q
            order_placed = False
            counter = 0

        for demand in day_demand:  # Iterate over each demand value for the current day
            if inventory - demand >= 0:
                data['units_sold'].append(demand)
                inventory -= demand
            else:
                data['units_sold'].append(inventory)
                data['units_lost'].append(demand - inventory)
                inventory = 0
                stock_out += 1

            data['inv_level'].append(inventory)

    return data

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

products = [Product(i) for i in range(1, 5)]
results = {}

for product in products:
    profit_list = []
    lost_orders_list = []
    reorder_quantities = []
    for _ in range(1000):
        data = MCS(Q=10000, product=product)
        profit = profit_calculation(data, product)
        profit_list.append(profit)
        lost_orders_list.append(len(data['units_lost']) / 365)
        reorder_quantities.extend(data['reorder_quantities'])
    results[f'Pr{product.i}'] = {
        'profits': profit_list,
        'mean_profit': np.mean(profit_list),
        'std_dev_profit': np.std(profit_list),
        'lost_order_proportion': np.mean(lost_orders_list),
        'reorder_quantity': np.mean(reorder_quantities),
        'average_safety_stock': np.mean(safety_stocks)
    }

for key, value in results.items():
    print(f"Product {key}:")
    print(f"  Mean Profit: {value['mean_profit']}")
    print(f"  Profit Std Dev: {value['std_dev_profit']}")
    print(f"  Proportion of Lost Orders: {value['lost_order_proportion']}")
    print(f"  Average Reorder Quantity: {value['reorder_quantity']}\n")
    print(f"  Average Safety Stock: {value['average_safety_stock']}\n")

stop = timeit.default_timer()
print('Time: ', stop - start)    


Product Pr1:
  Mean Profit: 53562.715848547115
  Profit Std Dev: 57.546566184509324
  Proportion of Lost Orders: 360.22279452054795
  Average Reorder Quantity: 740.3944587842888

  Average Safety Stock: 15.182317161113351

Product Pr2:
  Mean Profit: 213796.96292612416
  Profit Std Dev: 43.7208700750343
  Proportion of Lost Orders: 345.99027671232875
  Average Reorder Quantity: 3886.0703478642486

  Average Safety Stock: 15.182317161113351

Product Pr3:
  Mean Profit: 129061.98528855586
  Profit Std Dev: 86.19589030362356
  Proportion of Lost Orders: 354.6782684931507
  Average Reorder Quantity: 2400.3504139585675

  Average Safety Stock: 15.182317161113351

Product Pr4:
  Mean Profit: 329184.6781704639
  Profit Std Dev: 442.6964311980076
  Proportion of Lost Orders: 360.2865287671233
  Average Reorder Quantity: 861.001641803255

  Average Safety Stock: 15.182317161113351

Time:  961.9680825999994


To optimize the safety stock calculation and obtain optimal solutions, the following improvements were performed:

- Dynamic Z-score Calculation: Instead of using a fixed Z-score like 1.65, the program dynamically calculates the Z-score based on the desired service level or confidence level. This allows for flexibility in setting the appropriate level of risk tolerance.

- Refined Demand Distribution: Rather than relying on a simple mixture of two normal distributions, used historical demand data to fit a more accurate demand distribution, such as a normal distribution or another appropriate probability distribution. This ensures that the demand variability is adequately captured.

- Simulation-based Optimization: simulation-based optimization algorithms to iteratively adjust parameters (e.g., reorder quantity, safety stock level) to maximize expected profit while meeting service level constraints and minimizing costs.


## safety_stock = z * std_dev_daily_sales * np.sqrt(lead_time)


In [3]:
start = timeit.default_timer()

def calculate_safety_stock(z, std_dev_daily_sales, lead_time):
    safety_stock = z * std_dev_daily_sales * np.sqrt(lead_time)
    return safety_stock

summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785]
}

class Product:
    def __init__(self, i):
        self.mean = np.mean([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.sd = np.std([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.mean_demand_lead_time = summary['Mean Demand (Lead Time)'][i - 1]
        self.std_dev_demand_lead_time = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.expected_demand_lead_time = summary['Expected Demand (Lead Time)'][i - 1]

def daily_demand(mean, sd, probability):
    if np.random.uniform(0, 1) > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS(Q, product, review_period=30, z_score=1.65):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability
    demand_lead = product.expected_demand_lead_time  # Corrected attribute name

    """
    - Importance sampling parameters
    - Define a new distribution that emphasizes extreme demand values
    - For simplicity, we used a mixture of two normal distributions
    - One centered around the mean and another centered around a high demand value
    """
    
    high_demand_value = 2 * mean  
    weights = [0.8, 0.2]  # Adjust weights to emphasize high demand values
    means = [mean, high_demand_value]
    sds = [sd, sd]  # Assuming same standard deviation for simplicity

    # max_daily_sales and avg_daily_sales 
    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]  # Corrected list comprehension
    max_daily_sales = np.max(daily_sales)
    avg_daily_sales = np.mean(daily_sales)

    # max_lead_time and avg_lead_time
    max_lead_time = lead_time
    avg_lead_time = np.mean(summary['Lead Time'])

    # max_daily_sales and avg_daily_sales 
    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
    std_dev_daily_sales = np.std(daily_sales)

    # safety stock 
    safety_stock = calculate_safety_stock(z_score, std_dev_daily_sales, lead_time)

   
    q = 0
    stock_out = 0
    counter = 0
    order_placed = False
    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': [], 'reorder_quantities': []}  

    for day in range(1, 365):
        
        day_demand = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]  
        if day_demand != 0:  
            data['daily_demand'].append(day_demand)

        if day % review_period == 0:
           
            q = max(0, safety_stock + demand_lead - inventory)
            q = max(q, int(0.2 * mean))  # Minimum order quantity
            order_placed = True
            data['orders'].append(q)
            data['reorder_quantities'].append(q)  # Store reorder quantity

        if order_placed:
            counter += 1

        if counter == lead_time:
            inventory += q
            order_placed = False
            counter = 0

        for demand in day_demand:  
            if inventory - demand >= 0:
                data['units_sold'].append(demand)
                inventory -= demand
            else:
                data['units_sold'].append(inventory)
                data['units_lost'].append(demand - inventory)
                inventory = 0
                stock_out += 1

            data['inv_level'].append(inventory)

    return data

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

products = [Product(i) for i in range(1, 5)]
results = {}
for product in products:
    profit_list = []
    lost_orders_list = []
    reorder_quantities = []
    for _ in range(1000):
        data = MCS(Q=10000, product=product)
        profit = profit_calculation(data, product)
        profit_list.append(profit)
        lost_orders_list.append(len(data['units_lost']) / 365)
        reorder_quantities.extend(data['reorder_quantities'])
    results[f'Pr{product.i}'] = {
        'profits': profit_list,
        'mean_profit': np.mean(profit_list),
        'std_dev_profit': np.std(profit_list),
        'lost_order_proportion': np.mean(lost_orders_list),
        'reorder_quantity': np.mean(reorder_quantities),
        'average_safety_stock': np.mean(safety_stocks)
    }

for key, value in results.items():
    print(f"Product {key}:")
    print(f"  Mean Profit: {value['mean_profit']}")
    print(f"  Profit Std Dev: {value['std_dev_profit']}")
    print(f"  Proportion of Lost Orders: {value['lost_order_proportion']}")
    print(f"  Average Reorder Quantity: {value['reorder_quantity']}\n")
    print(f"  Average Safety Stock: {value['average_safety_stock']}\n")
    
stop = timeit.default_timer()
print('Time: ', stop - start) 

Product Pr1:
  Mean Profit: 52906.33426234104
  Profit Std Dev: 42.97893815513405
  Proportion of Lost Orders: 360.30736986301366
  Average Reorder Quantity: 717.9607730395575

  Average Safety Stock: 15.182317161113351

Product Pr2:
  Mean Profit: 213974.17721501354
  Profit Std Dev: 41.39206010088078
  Proportion of Lost Orders: 345.9274630136986
  Average Reorder Quantity: 3904.3252752894473

  Average Safety Stock: 15.182317161113351

Product Pr3:
  Mean Profit: 125007.98802269816
  Profit Std Dev: 64.52736627409199
  Proportion of Lost Orders: 355.0554657534247
  Average Reorder Quantity: 2285.7223621078856

  Average Safety Stock: 15.182317161113351

Product Pr4:
  Mean Profit: 312170.5248981452
  Profit Std Dev: 209.76535949520226
  Proportion of Lost Orders: 360.5135068493151
  Average Reorder Quantity: 800.1983831570827

  Average Safety Stock: 15.182317161113351

Time:  953.3070233999997


### dynamic Z-score based on desired service level

In [2]:
start = timeit.default_timer()

def calculate_safety_stock(service_level, std_dev_daily_sales, lead_time):
    z = st.norm.ppf(service_level)  # dynamic Z-score based on desired service level
    safety_stock = z * std_dev_daily_sales * np.sqrt(lead_time)
    return safety_stock

summary = {
    'Purchase Cost': [12, 7, 6, 37],
    'Lead Time': [9, 6, 15, 12],
    'Size': [0.57, 0.05, 0.53, 1.05],
    'Selling Price': [16.10, 8.60, 10.20, 68],
    'Starting Stock': [2750, 22500, 5200, 1400],
    'Ch': [0.2 * 12, 0.2 * 7, 0.2 * 6, 0.2 * 37],
    'Co': [1000, 1200, 1000, 1200],
    'Probability': [0.76, 1.00, 0.70, 0.23],
    'Mean Demand (Lead Time)': [103.50, 648.55, 201.68, 150.06],
    'Std. Dev. of Demand (Lead Time)': [37.32, 26.45, 31.08, 3.21],
    'Expected Demand (Lead Time)': [705, 3891, 2266, 785]
}

class Product:
    def __init__(self, i):
        self.mean = np.mean([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.sd = np.std([np.log(j) for j in [summary['Expected Demand (Lead Time)'][i - 1]] if j > 0])
        self.i = i
        self.unit_cost = summary['Purchase Cost'][i - 1]
        self.lead_time = summary['Lead Time'][i - 1]
        self.size = summary['Size'][i - 1]
        self.selling_price = summary['Selling Price'][i - 1]
        self.holding_cost = summary['Ch'][i - 1]
        self.ordering_cost = summary['Co'][i - 1]
        self.probability = summary['Probability'][i - 1]
        self.starting_stock = summary['Starting Stock'][i - 1]
        self.mean_demand_lead_time = summary['Mean Demand (Lead Time)'][i - 1]
        self.std_dev_demand_lead_time = summary['Std. Dev. of Demand (Lead Time)'][i - 1]
        self.expected_demand_lead_time = summary['Expected Demand (Lead Time)'][i - 1]

def daily_demand(mean, sd, probability):
    if np.random.uniform(0, 1) > probability:
        return 0
    else:
        return np.exp(np.random.normal(mean, sd))

def MCS(Q, product, review_period=30, service_level=0.95):
    inventory = product.starting_stock
    mean = product.mean
    sd = product.sd
    lead_time = product.lead_time
    probability = product.probability
    demand_lead = product.expected_demand_lead_time

     """
    - Importance sampling parameters
    - Define a new distribution that emphasizes extreme demand values
    - For simplicity, we used a mixture of two normal distributions
    - One centered around the mean and another centered around a high demand value
    """
    high_demand_value = 2 * mean  
    weights = [0.8, 0.2]  # Adjust weights to emphasize high demand values
    means = [mean, high_demand_value]
    sds = [sd, sd]  # Assuming same standard deviation for simplicity
    
    daily_sales = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
    max_daily_sales = np.max(daily_sales)
    avg_daily_sales = np.mean(daily_sales)
    std_dev_daily_sales = np.std(daily_sales)

    safety_stock = calculate_safety_stock(service_level, std_dev_daily_sales, lead_time)

    q = 0
    stock_out = 0
    counter = 0
    order_placed = False
    data = {'inv_level': [], 'daily_demand': [], 'units_sold': [], 'units_lost': [], 'orders': [], 'reorder_quantities': []}

    for day in range(1, 365):
        day_demand = [np.random.normal(means[i], sds[i]) for i in np.random.choice(len(weights), size=365, p=weights)]
        if day_demand != 0:
            data['daily_demand'].append(day_demand)

        if day % review_period == 0:
            q = max(0, safety_stock + demand_lead - inventory)
            q = max(q, int(0.2 * mean))
            order_placed = True
            data['orders'].append(q)
            data['reorder_quantities'].append(q)

        if order_placed:
            counter += 1

        if counter == lead_time:
            inventory += q
            order_placed = False
            counter = 0

        for demand in day_demand:
            if inventory - demand >= 0:
                data['units_sold'].append(demand)
                inventory -= demand
            else:
                data['units_sold'].append(inventory)
                data['units_lost'].append(demand - inventory)
                inventory = 0
                stock_out += 1

            data['inv_level'].append(inventory)

    return data, safety_stock

def profit_calculation(data, product):
    unit_cost = product.unit_cost
    selling_price = product.selling_price
    holding_cost = product.holding_cost
    order_cost = product.ordering_cost
    size = product.size
    days = 365

    revenue = sum(data['units_sold']) * selling_price
    Co = len(data['orders']) * order_cost
    Ch = sum(data['inv_level']) * holding_cost * size / days
    cost = sum(data['orders']) * unit_cost

    profit = revenue - cost - Co - Ch

    return profit

products = [Product(i) for i in range(1, 5)]
results = {}

n_simulations = 1000
num_cores = -1  # Use all available cores

for product in products:
    results_list = Parallel(n_jobs=num_cores)(
        delayed(MCS)(Q=10000, product=product) for _ in range(n_simulations)
    )

    
    profit_list = [profit_calculation(result[0], product) for result in results_list]
    lost_orders_list = [len(result[0]['units_lost']) / 365 for result in results_list]
    reorder_quantities = [rq for result in results_list for rq in result[0]['reorder_quantities']]
    safety_stocks = [result[1] for result in results_list]

    results[f'Pr{product.i}'] = {
        'profits': profit_list,
        'mean_profit': np.mean(profit_list),
        'std_dev_profit': np.std(profit_list),
        'lost_order_proportion': np.mean(lost_orders_list),
        'reorder_quantity': np.mean(reorder_quantities),
        'average_safety_stock': np.mean(safety_stocks)
    }


for key, value in results.items():
    print(f"Product {key}:")
    print(f"  Mean Profit: {value['mean_profit']}")
    print(f"  Profit Std Dev: {value['std_dev_profit']}")
    print(f"  Proportion of Lost Orders: {value['lost_order_proportion']}")
    print(f"  Average Reorder Quantity: {value['reorder_quantity']}")
    print(f"  Average Safety Stock: {value['average_safety_stock']}\n")

stop = timeit.default_timer()
print('Time: ', stop - start)    

Product Pr1:
  Mean Profit: 52903.43532873124
  Profit Std Dev: 44.602192256919906
  Proportion of Lost Orders: 360.305402739726
  Average Reorder Quantity: 717.9138556795073
  Average Safety Stock: 12.913855679507185

Product Pr2:
  Mean Profit: 213977.55881193286
  Profit Std Dev: 40.27897399471167
  Proportion of Lost Orders: 345.9366657534246
  Average Reorder Quantity: 3904.3022897184455
  Average Safety Stock: 13.302289718445135

Product Pr3:
  Mean Profit: 125004.67217180431
  Profit Std Dev: 62.3052419464203
  Proportion of Lost Orders: 355.05380273972605
  Average Reorder Quantity: 2285.6547599948044
  Average Safety Stock: 19.65475999480457

Product Pr4:
  Mean Profit: 312174.76254272525
  Profit Std Dev: 206.82948508926043
  Proportion of Lost Orders: 360.5164630136986
  Average Reorder Quantity: 800.1823171611134
  Average Safety Stock: 15.182317161113351

Time:  1567.7489869
